In [6]:
# 热传导
from collections import defaultdict
import pandas as pd
from tqdm import tqdm

# 迭代的轮数。一轮迭代包括用户->物品和物品->用户两次扩散
step=5


In [7]:
data=[
    ['A','b'],
    ['A','d'],
    ['B','a'],
    ['B','b'],
    ['B','c'],
    ['C','a'],
    ['C','b'],
    ['C','d'],
    ['D','a'],
    ['D','e']
]

In [8]:
# user->item的映射
# key：user
# value：item的set
train_data = {}

for user, item in data:
    train_data.setdefault(user,set())
    train_data[user].add(item)
train_data

{'A': {'b', 'd'}, 'B': {'a', 'b', 'c'}, 'C': {'a', 'b', 'd'}, 'D': {'a', 'e'}}

In [9]:
# 建立item_user倒排表
item_users = dict()
for u, items in train_data.items():
    for i in items:
        if i not in item_users:
            item_users[i] = set()
        item_users[i].add(u)
item_users

{'b': {'A', 'B', 'C'},
 'd': {'A', 'C'},
 'a': {'B', 'C', 'D'},
 'c': {'B'},
 'e': {'D'}}

In [10]:
user_item = train_data
rec_item = defaultdict(list)
for u in tqdm(user_item):
    print(u)
    item_cnt = defaultdict(int)
    user_cnt = defaultdict(int)
    for i in user_item[u]:
        item_cnt[i] = 1
    for k in range(step*2):
        if k % 2 == 1:
            # 用户扩散项目
            item_cnt = defaultdict(int)
            for user in user_cnt:
                for item in user_item[user]:
                    item_cnt[item] += user_cnt[user] / len(item_users[item])
#             print(item_cnt)

        else:
            # 项目扩散用户
            user_cnt = defaultdict(int)
            for item in item_cnt:
                for user in item_users[item]:
                    user_cnt[user] += item_cnt[item] / (len(user_item[user]))
#             print(user_cnt)

    res = ((pd.DataFrame(item_cnt, index=[0])).T).sort_values([0], ascending=[0]).index.tolist()

    rec_item[u] = [i for i in res if i not in user_item[u]]
    print(item_cnt)
    
    
    

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 445.67it/s]

A
defaultdict(<class 'int'>, {'b': 0.5240636351801047, 'd': 0.5355584252591069, 'a': 0.47974814878321403, 'c': 0.5010740550221003, 'e': 0.41637374256973025})
B
defaultdict(<class 'int'>, {'b': 0.7031438865920675, 'd': 0.7015388735679265, 'a': 0.6980340876856508, 'c': 0.7063539126403494, 'e': 0.6870344173906415})
C
defaultdict(<class 'int'>, {'b': 0.8137913004453929, 'd': 0.8205891505359956, 'a': 0.7883473894562143, 'c': 0.8001956002641873, 'e': 0.7522100289590001})
D
defaultdict(<class 'int'>, {'e': 0.4882854176192653, 'a': 0.4205446694694237, 'b': 0.3750134289954106, 'd': 0.36528895096657016, 'c': 0.3944623850530915})
